
<h2>Stock Price Prediction</h2>

<p>Using Yahoo Finance API (yfinance) as well as Linear Regression to predict stock prices. 
Documentation for yfinance can be found <a href="https://pypi.org/project/yfinance/">Here</a></p>

<h2> Summary:</h2>
<ol>
    <li>Basic emplementation of Machine Learning in price prediction</li>
    <li>SVR: Support Vector Regression gave a higher convidence in predictions</li>
    <li>SVR model always under priced the predictions while linear regression always overpriced</li>
    

</ol>

In [1]:
## Library Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf

In [2]:
# retrieving multiple stocks with tickers: AAPL, MSFT, GOOG

data = yf.download(tickers='AAPL MSFT GOOG',
                  period='1y', #for 1 year
                  #interval='5m', # five minute intervals
                  group_by = 'ticker',
                  )

[*********************100%***********************]  3 of 3 completed


In [3]:
# seperating each ticker to it's own df.
# data variable has multiindex on columns. and we can retrieve the first level as follows:
tickers = {ticker:data[ticker] for ticker in data.columns.levels [0]}
# so now data can be retrieved for each stock by using tickers.ticker as below
tickers['AAPL']





,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-30,80.135002,81.022499,79.687500,80.967499,80.274246,126743200
2020-01-31,80.232498,80.669998,77.072502,77.377502,76.714989,199588400
2020-02-03,76.074997,78.372498,75.555000,77.165001,76.504311,173985600
2020-02-04,78.827499,79.910004,78.407501,79.712502,79.029999,136616400
2020-02-05,80.879997,81.190002,79.737503,80.362503,79.674438,118826800
...,...,...,...,...,...,...
2021-01-25,143.070007,145.089996,136.539993,142.919998,142.919998,157611700
2021-01-26,143.600006,144.300003,141.369995,143.160004,143.160004,98390600
2021-01-27,143.429993,144.300003,140.410004,142.059998,142.059998,140843800


In [4]:
data.columns.levels[0]

Index(['AAPL', 'GOOG', 'MSFT'], dtype='object')

In [5]:
## Another method to retrieve stock information
apple = yf.Ticker("AAPL")


In [6]:
apple.actions # to show dividends and splits

,Dividends,Stock Splits
Date,,
1987-05-11,0.000536,0.0
1987-06-16,0.000000,2.0
1987-08-10,0.000536,0.0
1987-11-17,0.000714,0.0
1988-02-12,0.000714,0.0
...,...,...
2020-02-07,0.192500,0.0
2020-05-08,0.205000,0.0
2020-08-07,0.205000,0.0


In [7]:
apple.splits # show splits

Date
1987-06-16    2.0
2000-06-21    2.0
2005-02-28    2.0
2014-06-09    7.0
2020-08-31    4.0
Name: Stock Splits, dtype: float64

In [8]:
apple.dividends[-10:] # showing last 1o dividends

Date
2018-08-10    0.1825
2018-11-08    0.1825
2019-02-08    0.1825
2019-05-10    0.1925
2019-08-09    0.1925
2019-11-07    0.1925
2020-02-07    0.1925
2020-05-08    0.2050
2020-08-07    0.2050
2020-11-06    0.2050
Name: Dividends, dtype: float64

In [9]:
apple = tickers['AAPL']
apple


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-30,80.135002,81.022499,79.687500,80.967499,80.274246,126743200
2020-01-31,80.232498,80.669998,77.072502,77.377502,76.714989,199588400
2020-02-03,76.074997,78.372498,75.555000,77.165001,76.504311,173985600
2020-02-04,78.827499,79.910004,78.407501,79.712502,79.029999,136616400
2020-02-05,80.879997,81.190002,79.737503,80.362503,79.674438,118826800
...,...,...,...,...,...,...
2021-01-25,143.070007,145.089996,136.539993,142.919998,142.919998,157611700
2021-01-26,143.600006,144.300003,141.369995,143.160004,143.160004,98390600
2021-01-27,143.429993,144.300003,140.410004,142.059998,142.059998,140843800


In [10]:
# Lokking only at the Adj Close column
apple = apple[['Adj Close']]
apple.head()

,Adj Close
Date,
2020-01-30,80.274246
2020-01-31,76.714989
2020-02-03,76.504311
2020-02-04,79.029999
2020-02-05,79.674438


In [11]:
# create a prediction for 30 days in the future. shifting the data up by 30 days and adding it to the df

forecast_days = 30
apple['prediction'] = apple['Adj Close'].shift(-forecast_days)

/Users/spydermachome/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
print(apple.head())
print(apple.tail())

            Adj Close  prediction
Date                             
2020-01-30  80.274246   69.061012
2020-01-31  76.714989   60.176525
2020-02-03  76.504311   62.822491
2020-02-04  79.029999   61.284599
2020-02-05  79.674438   60.815033
             Adj Close  prediction
Date                              
2021-01-25  142.919998         NaN
2021-01-26  143.160004         NaN
2021-01-27  142.059998         NaN
2021-01-28  137.089996         NaN
2021-01-29  131.960007         NaN


In [13]:
# the data upto to the last thirty days will be used to train and test the model. The last thirty rows(days) will be predicted

X = np.array(apple.drop(['prediction'], 1))
X = X[:-forecast_days] # removing the last 30 days

#print(X)


In [14]:
y = np.array(apple.drop(['Adj Close'],1))
y = y[:-forecast_days]
#print(y)
print(f'X has length {len(X)}')
print(f'y has length of {len(y)}')

X has length 223
y has length of 223


In [15]:
X.shape

(223, 1)

## Modelling

In [16]:
# library imports for preprocessing and modelling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR


In [17]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X,y) 

In [18]:
lreg = LinearRegression()
svr = SVR(kernel='rbf', C=1e3, gamma=0.1)

In [19]:
lreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [20]:
lreg.score(X_test, y_test)

0.7096239081565121

In [21]:
svr.fit(X_train, y_train.ravel()) # using ravel to convert y from column vector to 1d array


SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [22]:
svr.score(X_test,y_test)

0.7914416971358357

In [23]:
# predicting the 30 days into the future
forecast_x = np.array(apple['Adj Close'][-forecast_days:])
lreg_prediction = lreg.predict(forecast_x.reshape(-1,1))
svr_prediction = svr.predict(forecast_x.reshape(-1,1))

In [24]:
lreg_prediction

array([[132.55186404],
       [133.38266914],
       [131.4783568 ],
       [132.94392793],
       [136.3511732 ],
       [135.49236456],
       [136.43518384],
       [140.84125548],
       [139.14229773],
       [138.06879049],
       [137.10729743],
       [134.04545296],
       [135.53902764],
       [131.42234258],
       [135.455017  ],
       [136.50986471],
       [133.64404506],
       [133.47602379],
       [135.42701345],
       [133.5787082 ],
       [131.92642777],
       [132.57053782],
       [136.49119093],
       [141.00927676],
       [143.06296509],
       [146.65689117],
       [146.88093378],
       [145.85408962],
       [141.21464559],
       [136.42585407]])

In [25]:
forecast_x

array([127.80999756, 128.69999695, 126.66000366, 128.22999573,
       131.88000488, 130.96000671, 131.97000122, 136.69000244,
       134.86999512, 133.72000122, 132.69000244, 129.41000366,
       131.00999451, 126.59999847, 130.91999817, 132.05000305,
       128.97999573, 128.80000305, 130.88999939, 128.91000366,
       127.13999939, 127.83000183, 132.02999878, 136.86999512,
       139.07000732, 142.91999817, 143.16000366, 142.05999756,
       137.08999634, 131.96000671])

In [26]:
svr_prediction

array([130.59365687, 122.69333654, 120.05858938, 128.389041  ,
       130.36839248, 117.21695272, 131.3212494 ,  97.23742573,
       107.2004643 , 124.60204467, 134.33942514, 113.01378119,
       117.94176198, 119.14567728, 116.64932888, 132.07400701,
       118.62003399, 121.25025615, 116.23196051, 119.64025885,
       126.77532778, 130.56835452, 131.89449027,  97.43395705,
       102.7070729 , 104.55317444, 104.55646375, 104.52236618,
        97.82157891, 131.22081317])

In [27]:
future_prices = apple[-forecast_days:].copy(deep=True)

In [28]:
future_prices["SVR_Prediction"] = svr_prediction
future_prices["Linear_regression"] = lreg_prediction

In [29]:
future_prices

,Adj Close,prediction,SVR_Prediction,Linear_regression
Date,,,,
2020-12-16,127.809998,NaN,130.593657,132.551864
2020-12-17,128.699997,NaN,122.693337,133.382669
2020-12-18,126.660004,NaN,120.058589,131.478357
2020-12-21,128.229996,NaN,128.389041,132.943928
2020-12-22,131.880005,NaN,130.368392,136.351173
2020-12-23,130.960007,NaN,117.216953,135.492365
2020-12-24,131.970001,NaN,131.321249,136.435184
2020-12-28,136.690002,NaN,97.237426,140.841255
2020-12-29,134.869995,NaN,107.200464,139.142298
